In [71]:
# get data from postgresql
import psycopg2

conn = psycopg2.connect(
    host="192.168.1.144",
    port="5432",
    database="majiang",
    user="postgres",
    password="pgpassword")



In [92]:
# 计算每家店每天的销售额
# conn = psycopg2.connect(
#     host="192.168.1.144",
#     port="5432",
#     database="majiang",
#     user="postgres",
#     password="pgpassword")

cur = conn.cursor()
cur.execute("SELECT store_name, SUM(CAST(price AS FLOAT)) as revenue FROM public.fourfriends \
            WHERE store_name LIKE '%武汉%'\
            AND date = '2024616'\
            and hour >= '10' and hour <= '22'\
            GROUP BY store_name \
            ORDER BY revenue desc;")
store_prices = cur.fetchall()
cur.close()
print(store_prices)


[('【武汉-云尚国际时尚中心店】', 5600.0), ('【武汉-普提金国际广场店】', 4578.0), ('【武汉-绿地熙街店】', 4494.0), ('【武汉-江汉路大洋百货店】', 4340.0), ('【武汉-东部购物公园店】', 4172.0), ('【武汉-武商城市奥莱店】', 3920.0), ('【武汉-和平大道奥山世纪广场商场店】', 3920.0), ('【武汉-佳园路店】', 3780.0), ('【武汉-汉正街美奇国际店】', 3640.0), ('【武汉-江汉路宝丽金店】', 3500.0), ('【武汉-金地自在城店】', 3360.0), ('【武汉-黄陂滠口店】', 3332.0), ('【武汉-江南花山郡东门店】', 3304.0), ('【武汉-黎黄陂路店】', 3290.0), ('【武汉-光谷创业街店】', 3290.0), ('【武汉-石牌岭鸿岭花园店】', 3290.0), ('【武汉-盘龙城奥特莱斯店】', 3248.0), ('【武汉-江汉路万达店】', 3248.0), ('【武汉-马湖商业街店】', 3248.0), ('【武汉-海林购物中心店】', 3150.0), ('【武汉-武广中山广场店】', 3150.0), ('【武汉-天成美景商业街店】', 3150.0), ('【武汉-大武汉1911店】', 3122.0), ('【武汉-水岸国际店】', 3122.0), ('【武汉-永旺泰富城15号楼店】', 3094.0), ('【武汉-永旺泰富城14号楼店】', 3094.0), ('【武汉-湘隆时代广场店】', 3080.0), ('【武汉-汉街龙源国际店】', 3066.0), ('【武汉-软件新城花城汇店】', 3010.0), ('【武汉-欧亚达汉阳国际广场店】', 3010.0), ('【武汉-奥山青扬六和店】', 2954.0), ('【武汉-光谷金融港店】', 2940.0), ('【武汉-古田龙湖春江郦城店】', 2940.0), ('【武汉-广信万汇城店】', 2926.0), ('【武汉-光谷鲁巷长江职院店】', 2884.0), ('【武汉-京华国际店】', 2870.0), ('【武汉-汉商银座店】', 2828.0), ('【武汉-鼓楼洞店】', 2814.0), ('【武汉